# Imports

In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

# For Preprocessing
!pip install -q -U datasets

import json
import pandas as pd
import json
import random
from datasets import Dataset, load_dataset
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

!pip install -q -U torch torchvision torchaudio fastai
!pip install -q -U bitsandbytes
!pip install -q -U transformers
!pip install -q -U peft
!pip install -q -U accelerate
!pip install -q -U tokenizers
!pip install -q -U datasets
!pip install -q -U evaluate
!pip install -q -U rouge_score
!pip install -q -U bert_score
!pip install -q -U loralib einops xformers

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

import os
import bitsandbytes as bnb
import pandas as pd
from peft import (
    LoraConfig,
    PeftConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from transformers import (
    AutoConfig,
    BitsAndBytesConfig,
)
import evaluate

import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize

from textwrap import fill
from transformers import pipeline
from sortedcontainers import SortedList
from tqdm.notebook import tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 143.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 95.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 75.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


# Load Prepared Documents

These are 10 randomly chosen short stories from the SQuALITY test set, with 2 questions each, for 20 total samples.

In [3]:
import pickle

from google.colab import drive
drive.mount('/content/drive')

retrieve_test_samples = []

try:
  if not retrieve_test_samples:
    file_name = "hallucination_study_test_samples-2025-08-02_171427.pkl"
    with open(f"/content/drive/MyDrive/DS266/project/analysis/data/{file_name}", "rb") as file:  # "rb" for read binary
      retrieve_test_samples = pickle.load(file)
except Exception as e:
  print(f"Error: {e}")


print("\n")
print("Test samples:", len(retrieve_test_samples), "\n")
retrieve_test_document = retrieve_test_samples[0]
retrieve_document_length = len(retrieve_test_document[0].split())
retrieve_document_text = fill(retrieve_test_document[0], width=80)
retrieve_question_length = len(retrieve_test_document[1].split())
retrieve_question_text = retrieve_test_document[1]
print(f"\033[1mDocument ({retrieve_document_length} words):\033[0m {retrieve_document_text}", "\n")
print(f"\033[1mQuestion ({retrieve_question_length} words):\033[0m {retrieve_question_text}", "\n")


Mounted at /content/drive


Test samples: 5 

Document (401 words): HOME IS WHERE YOU LEFT IT By ADAM CHASE [Transcriber Note: This etext was
produced from Amazing Stories February 1957. Extensive research did not uncover
any evidence that the U.S. copyright on this publication was renewed.] The
chance of mass slaughter was their eternal nightmare. How black is the blackest
treachery? Is the most callous traitor entitled to mercy? Steve pondered these
questions. His decision? That at times the villain should possibly be spoken of
as a hero. Only the shells of deserted mud-brick houses greeted Steve Cantwell
when he reached the village. He poked around in them for a while. The desert
heat was searing, parching, and the Sirian sun gleamed balefully off the blades
of Steve's unicopter, which had brought him from Oasis City, almost five hundred
miles away. He had remembered heat from his childhood here on Sirius' second
planet with the Earth colony, but not heat like this. It was like a ma

In [4]:
# Load the trained model.
from google.colab import drive
drive.mount('/content/drive')

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    load_4bit_use_double_quant=True,
)

model_name = "t5gemma_augmented-2025-07-29_145811"

model_quantized = AutoModelForSeq2SeqLM.from_pretrained(
    f"/content/drive/MyDrive/DS266/project/models/{model_name}",
    quantization_config=bnb_config,
    device_map={"": 0})

# Tokenizer was unchanged.
tokenizer_quantized = AutoTokenizer.from_pretrained("google/t5gemma-2b-2b-ul2-it")
model_quantized = prepare_model_for_kbit_training(model_quantized)

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["k_proj", "v_proj", "q_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_2_SEQ_LM",
    modules_to_save=["lm_head"]
)

model_quantized = get_peft_model(model_quantized, config)
print(model_quantized)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


config.json:   0%|          | 0.00/3.26k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/68.1k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/34.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/577 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): T5GemmaForConditionalGeneration(
      (model): T5GemmaModel(
        (encoder): T5GemmaEncoder(
          (embed_tokens): Embedding(256000, 2304, padding_idx=0)
          (norm): T5GemmaRMSNorm((2304,), eps=1e-06)
          (rotary_emb): T5GemmaRotaryEmbedding()
          (layers): ModuleList(
            (0-25): 26 x T5GemmaEncoderLayer(
              (self_attn): T5GemmaSelfAttention(
                (q_proj): lora.Linear4bit(
                  (base_layer): Linear4bit(in_features=2304, out_features=2048, bias=False)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=2304, out_features=16, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_features=16, out_features=2048, bias=False)
             

In [5]:
# Functions to generate output.
from textwrap import fill
from tqdm.notebook import tqdm

MAX_SEQUENCE_LENGTH = 1024

# Hyperparameters
generate_kwargs = {
    "num_beams": 3,
    "do_sample": True,
    "no_repeat_ngram_size": 4,
    "max_length": 512
}

def get_sentences(text):
  return nltk.sent_tokenize(text)

def generate_output_quantized(model, tokenizer, dataset, batch_size, max_length=MAX_SEQUENCE_LENGTH):
  formatted_inputs = []
  for document, question in zip(dataset["document"], dataset["question"]):
    formatted_inputs.append([{"role": "user", "content": f"Question: {question} Context: {document}"}])

  outputs = []
  #pbar = tqdm(range(int(len(formatted_inputs) / batch_size) + 1))
  for i in range(int(len(formatted_inputs) / batch_size) + 1):
    start_i, end_i = i * batch_size, (i + 1) * batch_size
    if start_i >= len(formatted_inputs):
        break

    #pbar.set_description_str(f"Samples {start_i} to {end_i}")

    input_ids = tokenizer.apply_chat_template(
        formatted_inputs[start_i:end_i],
        return_tensors="pt",
        return_dict=True,
        add_generation_prompt=True,
        max_length=max_length,
        padding=True,
        truncation=True,).to("cuda")
    encoded_output = model.generate(**input_ids, **generate_kwargs)
    generated_sentences = tokenizer.batch_decode(encoded_output,
                                                     skip_special_tokens=True,
                                                     clean_up_tokenization_spaces=False)
    outputs.extend(generated_sentences)

  return outputs

def generate_and_print_output(samples):
  for i, (document, question) in enumerate(samples, start=1):
    document = fill(document, width=80)
    summary = generate_output_quantized(
      model_quantized,
      tokenizer_quantized,
      { "document": [document], "question": [question] },
      16)
    response = fill(summary[0], width=80)
    d_sentence_count = len(get_sentences(document))
    r_sentence_count = len(get_sentences(response))
    print("\n")
    print("----------------------------------------------------------------\n")
    print(f"\033[1mDocument {i} ({d_sentence_count} sentences):\033[0m {document}", "\n")
    print(f"\033[1mQuestion {i}:\033[0m {question}", "\n")
    print(f"\033[1mResponse {i} ({r_sentence_count} sentences):\033[0m {response}", "\n")


In [6]:
generate_and_print_output(retrieve_test_samples)



----------------------------------------------------------------

Document 1 (32 sentences): HOME IS WHERE YOU LEFT IT By ADAM CHASE [Transcriber Note: This etext was
produced from Amazing Stories February 1957. Extensive research did not uncover
any evidence that the U.S. copyright on this publication was renewed.] The
chance of mass slaughter was their eternal nightmare. How black is the blackest
treachery? Is the most callous traitor entitled to mercy? Steve pondered these
questions. His decision? That at times the villain should possibly be spoken of
as a hero. Only the shells of deserted mud-brick houses greeted Steve Cantwell
when he reached the village. He poked around in them for a while. The desert
heat was searing, parching, and the Sirian sun gleamed balefully off the blades
of Steve's unicopter, which had brought him from Oasis City, almost five hundred
miles away. He had remembered heat from his childhood here on Sirius' second
planet with the Earth colony, but not heat 